See how groundtruth overlaps in the train set.
（iou>0, base and all for all train set scenes）

In [6]:
import os
import random
import numpy as np
DATA_PATH = os.path.join("/data2/wufang/sdcot/SDCoT/scannet/scannet_train_detection_data")
import sys
sys.path.append('/data2/wufang/sdcot/SDCoT/utils/')
from nms import nms_2d_faster, nms_3d_faster, nms_3d_faster_samecls
ALL_SCENE_NAMES = list(set([os.path.basename(x)[0:12] for x in os.listdir(DATA_PATH) if x.startswith('scene')]))

scene_name = random.choice(ALL_SCENE_NAMES)

In [7]:
NUM_BASE_CLASSES = 14
BASE_TYPES = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door', 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink']
BASE_NYUIDS = np.array([36, 4, 10, 3, 5, 12, 16, 14, 8, 39, 11, 24, 28, 34])

NUM_NOVEL_CLASSES = 4
NOVEL_TYPES = [ 'sofa', 'table', 'toilet', 'window']
NOVEL_NYUIDS = np.array([6, 7, 33, 9])



In [8]:
def generate_sample(ins):
    x=ins[0]
    y=ins[1]
    z=ins[2]
    l=ins[3]
    w=ins[4]
    h=ins[5]
    cla=ins[6]
    heading=0
    probility=1
    return [x,y,z,l,w,h,heading,cla,probility]


In [9]:
def generate_input(gt_bbox_new):
    x1 = gt_bbox_new[:, 0]
    y1 = gt_bbox_new[:, 1]
    z1 = gt_bbox_new[:, 2]
    x2 = gt_bbox_new[:, 0] + gt_bbox_new[:, 3]  # x2 = x1 + l
    y2 = gt_bbox_new[:, 1] + gt_bbox_new[:, 4]  # y2 = y1 + w
    z2 = gt_bbox_new[:, 2] + gt_bbox_new[:, 5]  # z2 = z1 + h
    heading = gt_bbox_new[:, 6]
    score = gt_bbox_new[:, 8]  # confidence score
    clss = gt_bbox_new[:, 7]  # class

    # each row is a bounding box: x1, y1, z1, x2, y2, z2, score, cls
    boxes = np.column_stack((x1, y1, z1, x2, y2, z2, heading, score, clss))
    return boxes

In [10]:
for overlap_threshold in [0.25, 0.5, 0.65, 0.75, 0.85, 0.9]:
    base_num=0
    all_num=0
    count_base = 0
    count_all = 0
    for scene_name in ALL_SCENE_NAMES:
        gt_bboxes = np.load(os.path.join(DATA_PATH, scene_name) + '_bbox.npy')
        bbox_mask_base = np.in1d(gt_bboxes[:, -1], BASE_NYUIDS)
        gt_bboxes_base = gt_bboxes[bbox_mask_base, :]
        gt_bbox_base_new = []
        for ins in gt_bboxes_base:
            gt_bbox_base_new.append(generate_sample(ins))

        gt_bbox_base_new = np.array(gt_bbox_base_new)
        if gt_bbox_base_new.size == 0:
            continue
        boxes_base = generate_input(gt_bbox_base_new)

        # do nms
        picked_indices_base = nms_3d_faster_samecls(boxes_base, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes_base = boxes_base[picked_indices_base]

        # all
        gt_bbox_new = []
        for ins in gt_bboxes:
            gt_bbox_new.append(generate_sample(ins))

        gt_bbox_new = np.array(gt_bbox_new)
        if gt_bbox_new.size == 0:
            continue
        boxes = generate_input(gt_bbox_new)

        picked_indices = nms_3d_faster_samecls(boxes, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes = boxes[picked_indices]
        count_base += boxes_base.shape[0]-selected_boxes_base.shape[0]
        base_num+=boxes_base.shape[0]
        count_all += boxes.shape[0]-selected_boxes.shape[0]
        all_num+=boxes.shape[0]
    precent_base = count_base/base_num
    precent_all = count_all/all_num
    print('overlap_threshold: {}, base: {}, all: {}, percent_base: {}, precent_all: {}'.format(overlap_threshold, count_base, count_all, precent_base, precent_all))


overlap_threshold: 0.25, base: 328, all: 369, percent_base: 0.02537324978726696, precent_all: 0.023501687790586586
overlap_threshold: 0.5, base: 111, all: 114, percent_base: 0.008586679043861685, precent_all: 0.007260684032864149
overlap_threshold: 0.65, base: 38, all: 38, percent_base: 0.0029395838168175138, precent_all: 0.0024202280109547163
overlap_threshold: 0.75, base: 9, all: 9, percent_base: 0.000696217219772569, precent_all: 0.0005732118973313802
overlap_threshold: 0.85, base: 2, all: 2, percent_base: 0.00015471493772723757, precent_all: 0.0001273804216291956
overlap_threshold: 0.9, base: 0, all: 0, percent_base: 0.0, precent_all: 0.0


In [11]:
NYU40IDS = np.array([36, 4, 10, 3, 5, 12, 16, 14, 8, 39, 11, 24, 28, 34, 6, 7, 33, 9]) #the corresponding NYU40 ids of interested object class
TYPE_WHITELIST = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door', 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink', 'sofa', 'table', 'toilet', 'window']

In [12]:
from collections import defaultdict

# Create a dictionary mapping class IDs to types
id_to_type = dict(zip(NYU40IDS, TYPE_WHITELIST))

for overlap_threshold in [0.25, 0.5, 0.65, 0.75, 0.85, 0.9]:
    base_num=0
    all_num=0
    count_base = 0
    count_all = 0
    class_counts_base = defaultdict(int)
    class_counts_all=defaultdict(int)
    class_all = defaultdict(int)
    class_base = defaultdict(int)
    for scene_name in ALL_SCENE_NAMES:
        gt_bboxes = np.load(os.path.join(DATA_PATH, scene_name) + '_bbox.npy')
        bbox_mask_base = np.in1d(gt_bboxes[:, -1], BASE_NYUIDS)
        gt_bboxes_base = gt_bboxes[bbox_mask_base, :]
        gt_bbox_base_new = []
        for ins in gt_bboxes_base:
            gt_bbox_base_new.append(generate_sample(ins))

        gt_bbox_base_new = np.array(gt_bbox_base_new)
        if gt_bbox_base_new.size == 0:
            continue
        boxes_base = generate_input(gt_bbox_base_new)

        # do nms
        picked_indices_base = nms_3d_faster_samecls(boxes_base, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes_base = boxes_base[picked_indices_base]


        boxes_set_base = set(map(tuple, boxes_base))
        boxes_set_base = np.array(list(boxes_set_base))
        
        for i in range(boxes_set_base.shape[0]):
            class_id = boxes_set_base[i][8]
            if class_id in id_to_type:
                class_type = id_to_type[class_id]
                class_base[class_type] += 1


        selected_boxes_set_base = set(map(tuple, selected_boxes_base))
        boxes_set_base = set(map(tuple, boxes_base))
        # Find the difference
        difference_base = boxes_set_base - selected_boxes_set_base

        # Convert the difference back to a numpy array
        difference_array_base = np.array(list(difference_base))
        if difference_array_base.size != 0:
            for i in range(difference_array_base.shape[0]):
                class_id = difference_array_base[i][8]
                if class_id in id_to_type:
                    class_type = id_to_type[class_id]
                    class_counts_base[class_type] += 1

        # all
        gt_bbox_new = []
        for ins in gt_bboxes:
            gt_bbox_new.append(generate_sample(ins))

        gt_bbox_new = np.array(gt_bbox_new)
        if gt_bbox_new.size == 0:
            continue
        boxes = generate_input(gt_bbox_new)

        picked_indices = nms_3d_faster_samecls(boxes, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes = boxes[picked_indices]

        # Convert the arrays to tuples and then to sets
        boxes_set = set(map(tuple, boxes))
        boxes_set = np.array(list(boxes_set))

        for i in range(boxes_set.shape[0]):
            class_id = boxes_set[i][8]
            if class_id in id_to_type:
                class_type = id_to_type[class_id]
                class_all[class_type] += 1


        selected_boxes_set = set(map(tuple, selected_boxes))
        boxes_set = set(map(tuple, boxes))
        # Find the difference
        difference = boxes_set - selected_boxes_set

        # Convert the difference back to a numpy array
        difference_array = np.array(list(difference))
        if difference_array.size != 0:
            for i in range(difference_array.shape[0]):
                class_id = difference_array[i][8]
                if class_id in id_to_type:
                    class_type = id_to_type[class_id]
                    class_counts_all[class_type] += 1

        count_base += boxes_base.shape[0]-selected_boxes_base.shape[0]
        base_num+=boxes_base.shape[0]
        count_all += boxes.shape[0]-selected_boxes.shape[0]
        all_num+=boxes.shape[0]
    precent_base = count_base/base_num
    precent_all = count_all/all_num
    print('overlap_threshold: {}, base: {}, all: {}, percent_base: {}, precent_all: {}'.format(overlap_threshold, count_base, count_all, precent_base, precent_all))

    # for class_type, count in class_base.items():
    #     print('Whole base class: {}, Count: {}'.format(class_type, count))
    # for class_type, count in class_all.items():
    #     print('Whole all class: {}, Count: {}'.format(class_type, count))
    # # ... existing code ...

    for class_type, count in class_counts_base.items():
        base_count = class_base[class_type]
        print('Base class: {}, Count: {}, Whole count: {}, Ratio: {}'.format(class_type, count, base_count, count / base_count))

    for class_type, count in class_counts_all.items():
        all_count = class_all[class_type]
        print('All class: {}, Count: {}, Whole count: {}, Ratio: {}'.format(class_type, count, all_count, count / all_count))



overlap_threshold: 0.25, base: 328, all: 369, percent_base: 0.02537324978726696, precent_all: 0.023501687790586586
Base class: curtain, Count: 10, Whole count: 292, Ratio: 0.03424657534246575
Base class: door, Count: 251, Whole count: 2026, Ratio: 0.12388943731490622
Base class: counter, Count: 7, Whole count: 216, Ratio: 0.032407407407407406
Base class: chair, Count: 23, Whole count: 4357, Ratio: 0.005278861602019738
Base class: desk, Count: 11, Whole count: 551, Ratio: 0.019963702359346643
Base class: showercurtain, Count: 1, Whole count: 116, Ratio: 0.008620689655172414
Base class: sink, Count: 5, Whole count: 390, Ratio: 0.01282051282051282
Base class: cabinet, Count: 9, Whole count: 1427, Ratio: 0.006306937631394534
Base class: refrigerator, Count: 2, Whole count: 186, Ratio: 0.010752688172043012
Base class: otherfurniture, Count: 7, Whole count: 1985, Ratio: 0.003526448362720403
Base class: bookshelf, Count: 2, Whole count: 300, Ratio: 0.006666666666666667
All class: curtain, Cou

9（psudu label）+9（gt）

In [16]:
NUM_BASE_CLASSES = 9
BASE_TYPES = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door']
BASE_NYUIDS = np.array([36, 4, 10, 3, 5, 12, 16, 14, 8 ])

NUM_NOVEL_CLASSES = 9
NOVEL_TYPES = [ 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink', 'sofa', 'table', 'toilet', 'window']
NOVEL_NYUIDS = np.array([39, 11, 24, 28, 34,6, 7, 33, 9])

id_to_type = dict(zip(BASE_NYUIDS, BASE_TYPES))

In [19]:
for overlap_threshold in [0.25, 0.5, 0.65, 0.75, 0.85, 0.9]:
    base_num=0
    all_num=0
    count_base = 0
    count_all = 0
    class_base = defaultdict(int)
    class_counts_base=defaultdict(int)
    for scene_name in ALL_SCENE_NAMES:
        gt_bboxes = np.load(os.path.join(DATA_PATH, scene_name) + '_bbox.npy')
        bbox_mask_base = np.in1d(gt_bboxes[:, -1], BASE_NYUIDS)
        bbox_mask_novel = np.in1d(gt_bboxes[:, -1], NOVEL_NYUIDS)
        gt_bboxes_base = gt_bboxes[bbox_mask_base, :]
        gt_bboxes_novel = gt_bboxes[bbox_mask_novel, :]

        gt_bbox_base_new = []
        for ins in gt_bboxes_base:
            inss=generate_sample(ins)
            inss[8]=0.9
            gt_bbox_base_new.append(inss)

        gt_bbox_base_new = np.array(gt_bbox_base_new)
        if gt_bbox_base_new.size == 0:
            continue
        boxes_base = generate_input(gt_bbox_base_new)

        gt_bboxes_novel_new = []
        for ins in gt_bboxes_novel:
            ins[6]=-1
            gt_bboxes_novel_new.append(generate_sample(ins))
        gt_bboxes_novel_new = np.array(gt_bboxes_novel_new)
        if gt_bboxes_novel_new.size == 0:
            continue
        boxes_novel = generate_input(gt_bboxes_novel_new)
        boxes = np.concatenate((boxes_base, boxes_novel), axis=0)
        # print(boxes.shape)
        # do nms
        picked_indices_base = nms_3d_faster_samecls(boxes, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes_base = boxes[picked_indices_base]

        # remove boxes where the last column is -1
        selected_boxes_base = selected_boxes_base[selected_boxes_base[:, -1] != -1]

        boxes_set_base = set(map(tuple, boxes_base))
        boxes_set_base = np.array(list(boxes_set_base))
        
        for i in range(boxes_set_base.shape[0]):
            class_id = boxes_set_base[i][8]
            if class_id in id_to_type:
                class_type = id_to_type[class_id]
                class_base[class_type] += 1


        selected_boxes_set_base = set(map(tuple, selected_boxes_base))
        boxes_set_base = set(map(tuple, boxes_base))
        # Find the difference
        difference_base = boxes_set_base - selected_boxes_set_base

        # Convert the difference back to a numpy array
        difference_array_base = np.array(list(difference_base))
        if difference_array_base.size != 0:
            for i in range(difference_array_base.shape[0]):
                class_id = difference_array_base[i][8]
                if class_id in id_to_type:
                    class_type = id_to_type[class_id]
                    class_counts_base[class_type] += 1
    print('overlap_threshold: {}'.format(overlap_threshold))
    for class_type, count in class_counts_base.items():
        base_count = class_base[class_type]
        print('Base class: {}, Count: {}, Whole count: {}, Ratio: {}'.format(class_type, count, base_count, count / base_count))


overlap_threshold: 0.25
Base class: curtain, Count: 12, Whole count: 291, Ratio: 0.041237113402061855
Base class: door, Count: 238, Whole count: 1986, Ratio: 0.11983887210473314
Base class: counter, Count: 4, Whole count: 216, Ratio: 0.018518518518518517
Base class: chair, Count: 23, Whole count: 4309, Ratio: 0.005337665351589696
Base class: desk, Count: 9, Whole count: 526, Ratio: 0.017110266159695818
Base class: cabinet, Count: 6, Whole count: 1412, Ratio: 0.00424929178470255
Base class: bookshelf, Count: 2, Whole count: 267, Ratio: 0.00749063670411985
overlap_threshold: 0.5
Base class: door, Count: 102, Whole count: 1986, Ratio: 0.0513595166163142
Base class: chair, Count: 2, Whole count: 4309, Ratio: 0.0004641448131817127
Base class: curtain, Count: 3, Whole count: 291, Ratio: 0.010309278350515464
overlap_threshold: 0.65
Base class: door, Count: 35, Whole count: 1986, Ratio: 0.017623363544813697
Base class: chair, Count: 1, Whole count: 4309, Ratio: 0.00023207240659085636
overlap_t

In [23]:
NYU40IDS = np.array([36, 4, 10, 3, 5, 12, 16, 14, 8, 39, 11, 24, 28, 34, 6, 7, 33, 9]) #the corresponding NYU40 ids of interested object class
TYPE_WHITELIST = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door', 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink', 'sofa', 'table', 'toilet', 'window']
id_to_type = dict(zip(NYU40IDS, TYPE_WHITELIST))

compute the number of class in instance and scene level

In [25]:
class_base=defaultdict(int)
scene_base=defaultdict(int)
for scene_name in ALL_SCENE_NAMES:
    gt_bboxes = np.load(os.path.join(DATA_PATH, scene_name) + '_bbox.npy')

    base=defaultdict(int)

    boxes_set_base = set(map(tuple, gt_bboxes))
    boxes_set_base = np.array(list(boxes_set_base))
        
    for i in range(boxes_set_base.shape[0]):
        class_id = boxes_set_base[i][6]
        if class_id in id_to_type:
            class_type = id_to_type[class_id]
            class_base[class_type] += 1
            base[class_type]+=1

    for class_type, count in base.items():
        if count==0:
            continue
        scene_base[class_type]+=1

    

for class_type, count in class_base.items():
    print('class: {}, inc: {}, scene: {}'.format(class_type, count, scene_base[class_type]))

class: curtain, inc: 292, scene: 193
class: chair, inc: 4357, scene: 798
class: bookshelf, inc: 300, scene: 172
class: picture, inc: 661, scene: 321
class: cabinet, inc: 1427, scene: 564
class: desk, inc: 551, scene: 327
class: otherfurniture, inc: 1985, scene: 890
class: window, inc: 928, scene: 609
class: bed, inc: 307, scene: 245
class: door, inc: 2026, scene: 874
class: counter, inc: 216, scene: 179
class: sink, inc: 390, scene: 322
class: refrigerator, inc: 186, scene: 177
class: table, inc: 1271, scene: 623
class: bathtub, inc: 113, scene: 113
class: showercurtain, inc: 116, scene: 106
class: toilet, inc: 201, scene: 189
class: sofa, inc: 406, scene: 279


find the best threshold for door class

In [26]:
NUM_BASE_CLASSES = 1
BASE_TYPES = [ 'door']
BASE_NYUIDS = np.array([8 ])

In [35]:
for overlap_threshold in [0.25, 0.5, 0.65, 0.75, 0.85, 0.9]:
    base_num=0
    all_num=0
    count_base = 0
    count_all = 0
    class_base = defaultdict(int)
    class_counts_base=defaultdict(int)
    for scene_name in ALL_SCENE_NAMES:
        gt_bboxes = np.load(os.path.join(DATA_PATH, scene_name) + '_bbox.npy')
        bbox_mask_base = np.in1d(gt_bboxes[:, -1], BASE_NYUIDS)
        gt_bboxes_base = gt_bboxes[bbox_mask_base, :]

        gt_bbox_base_new = []
        for ins in gt_bboxes_base:
            inss=generate_sample(ins)
            inss[8]=0.9
            gt_bbox_base_new.append(inss)

        gt_bbox_base_new = np.array(gt_bbox_base_new)
        if gt_bbox_base_new.size == 0:
            continue
        boxes_base = generate_input(gt_bbox_base_new)

        # print(boxes.shape)
        # do nms
        picked_indices_base = nms_3d_faster_samecls(boxes_base, overlap_threshold, old_type=False)

        # pick out the selected boxes
        selected_boxes_base = boxes_base[picked_indices_base]

        boxes_set_base = set(map(tuple, boxes_base))
        boxes_set_base = np.array(list(boxes_set_base))
        
        for i in range(boxes_set_base.shape[0]):
            class_id = boxes_set_base[i][8]
            if class_id in id_to_type:
                class_type = id_to_type[class_id]
                class_base[class_type] += 1


        selected_boxes_set_base = set(map(tuple, selected_boxes_base))
        boxes_set_base = set(map(tuple, boxes_base))
        # Find the difference
        difference_base = boxes_set_base - selected_boxes_set_base
        # print(len(difference_base))
        if len(difference_base)==6:
            print(scene_name, len(difference_base))

        # Convert the difference back to a numpy array
        difference_array_base = np.array(list(difference_base))
        if difference_array_base.size != 0:
            for i in range(difference_array_base.shape[0]):
                class_id = difference_array_base[i][8]
                if class_id in id_to_type:
                    class_type = id_to_type[class_id]
                    class_counts_base[class_type] += 1
    print('overlap_threshold: {}'.format(overlap_threshold))
    for class_type, count in class_counts_base.items():
        base_count = class_base[class_type]
        print('Base class: {}, Count: {}, Whole count: {}, Ratio: {}'.format(class_type, count, base_count, count / base_count))


scene0012_00 6
overlap_threshold: 0.25
Base class: door, Count: 246, Whole count: 2026, Ratio: 0.12142152023692004
overlap_threshold: 0.5
Base class: door, Count: 105, Whole count: 2026, Ratio: 0.051826258637709774
overlap_threshold: 0.65
Base class: door, Count: 37, Whole count: 2026, Ratio: 0.01826258637709773
overlap_threshold: 0.75
Base class: door, Count: 8, Whole count: 2026, Ratio: 0.003948667324777887
overlap_threshold: 0.85
Base class: door, Count: 2, Whole count: 2026, Ratio: 0.0009871668311944718
overlap_threshold: 0.9


In [39]:
from box_util import get_3d_box_depth

In [66]:
gt_bboxes = np.load(os.path.join(DATA_PATH, 'scene0012_00') + '_bbox.npy')
gt=[]

In [67]:
for i in range(gt_bboxes.shape[0]):
    bboxes = gt_bboxes[i]

    box3d = get_3d_box_depth(bboxes[3:6], 0, bboxes[0:3])
    if bboxes[6]==8:
        print(bboxes[6])
        gt.append(box3d)

8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0
8.0


In [70]:
gt

[array([[-1.5995943 ,  2.27363372,  1.63597012],
        [-1.5995943 ,  1.13791561,  1.63597012],
        [-1.73160201,  1.13791561,  1.63597012],
        [-1.73160201,  2.27363372,  1.63597012],
        [-1.5995943 ,  2.27363372,  0.01258051],
        [-1.5995943 ,  1.13791561,  0.01258051],
        [-1.73160201,  1.13791561,  0.01258051],
        [-1.73160201,  2.27363372,  0.01258051]]),
 array([[-1.59163165,  2.28307462,  1.73982739],
        [-1.59163165,  0.98218727,  1.73982739],
        [-1.72998261,  0.98218727,  1.73982739],
        [-1.72998261,  2.28307462,  1.73982739],
        [-1.59163165,  2.28307462,  0.02488232],
        [-1.59163165,  0.98218727,  0.02488232],
        [-1.72998261,  0.98218727,  0.02488232],
        [-1.72998261,  2.28307462,  0.02488232]]),
 array([[-1.55917817,  0.94972384,  1.68263149],
        [-1.55917817, -0.67188787,  1.68263149],
        [-1.72117299, -0.67188787,  1.68263149],
        [-1.72117299,  0.94972384,  1.68263149],
        [-1.5591